In [1]:
import glob
import pickle
import numpy
from music21 import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
def sequence(notes,n_vocab):
    """ Create input and output sequences """
    
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    sequence_length = 50
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
       
    n_patterns = len(network_input)     
    
    # Reshape the input into a format compatible with LSTM layers and normalize
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    n_vocab = len(set(notes))
    network_input = network_input / n_vocab
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input,network_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ Create network structure """
    model = Sequential()

    model.add(LSTM(
    500,
    return_sequences=True,
    time_major=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(500, return_sequences=True))
    model.add(Dropout(0.3)) 
    model.add(LSTM(500))
    model.add(Dense(250))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation("softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    
      # Load the weights to each node
    model.load_weights('/Users/leesurkis/Documents/GitHub/Music_Files/Generating_Techno_Music/weights-improvement-30-0.4634-bigger.hdf5')

    return model